01. SETUP
    - get dataset and tfrecord from roboflow
    - clone tensorflow model lib
    - get pretrained model

In [2]:
import os
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile
import pathlib
import tarfile
import urllib.request


In [1]:
DATASET_DIR = 'C:\\Users\\ardo\\Documents\\personal\\dev\\notebook\\test_count_1404\\datasets'
WORKSPACE_DIR = 'C:\\Users\\ardo\\Documents\\personal\\dev\\notebook\\test_count_1404\\workspace'

if not os.path.exists(DATASET_DIR):
    os.makedirs(DATASET_DIR)

if not os.path.exists(WORKSPACE_DIR):
    os.makedirs(WORKSPACE_DIR)

In [8]:
def download_and_unzip(url, extract_to='.'):
    http_response = urlopen(url)
    zipfile = ZipFile(BytesIO(http_response.read()))
    zipfile.extractall(path=extract_to)
    
download_and_unzip('https://app.roboflow.com/ds/9kWH42JFYv?key=g4wuao6Y47', DATASET_DIR)

In [3]:
# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
Updating files:  96% (2307/2395)
Updating files:  97% (2324/2395)
Updating files:  98% (2348/2395)
Updating files:  99% (2372/2395)
Updating files: 100% (2395/2395)
Updating files: 100% (2395/2395), done.


In [3]:
pretrained_url = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz'
pretrained_filename = 'ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz'
pretrained_foldername = 'ssd_resnet50_v1_fpn_640x640_coco17_tpu-8'

if not (os.path.exists(pretrained_foldername) or os.path.exists(pretrained_filename)):
    print('downloading..')
    urllib.request.urlretrieve(pretrained_url, pretrained_filename)
    print('extracting..')
    if pretrained_filename.endswith("tar.gz"):
        tar = tarfile.open(pretrained_filename, "r:gz")
        tar.extractall()
        tar.close()
    print('remove raw file..')
    os.remove(pretrained_filename)

print('done')

done


In [12]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [14]:
CONFIG_PATH = pretrained_foldername + '/pipeline.config'
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
config

{'model': ssd {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 640
       width: 640
     }
   }
   feature_extractor {
     type: "ssd_resnet50_v1_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 0.0004
         }
       }
       initializer {
         truncated_normal_initializer {
           mean: 0.0
           stddev: 0.03
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.997
         scale: true
         epsilon: 0.001
       }
     }
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matcher {
     argmax_matcher {
       matched_threshold: 0.5
       unmatched_threshold: 0.5
       ignore_thresholds: fa

In [15]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)

In [16]:
pipeline_config.model.ssd.num_classes = 2
pipeline_config.train_config.batch_size = 10
pipeline_config.train_config.fine_tune_checkpoint = pretrained_foldername + '/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= DATASET_DIR + '/train/bonggol_label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [DATASET_DIR + '/train/bonggol.tfrecord']
pipeline_config.eval_input_reader[0].label_map_path = DATASET_DIR + '/test/bonggol_label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [DATASET_DIR + '/test/bonggol.tfrecord']

In [17]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)

config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
config

{'model': ssd {
   num_classes: 2
   image_resizer {
     fixed_shape_resizer {
       height: 640
       width: 640
     }
   }
   feature_extractor {
     type: "ssd_resnet50_v1_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 0.0004
         }
       }
       initializer {
         truncated_normal_initializer {
           mean: 0.0
           stddev: 0.03
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.997
         scale: true
         epsilon: 0.001
       }
     }
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matcher {
     argmax_matcher {
       matched_threshold: 0.5
       unmatched_threshold: 0.5
       ignore_thresholds: fal

In [21]:
print("""python C:\\Users\\ardo\\Documents\\personal\\dev\\notebook\\test_count_1404\\models\\research\\object_detection\\model_main_tf2.py --model_dir= --pipeline_config_path={} --num_train_steps=100""".format(WORKSPACE_DIR,CONFIG_PATH))

python C:\Users\ardo\Documents\personal\dev\notebook\test_count_1404\models\research\object_detection\model_main_tf2.py --model_dir= --pipeline_config_path=C:\Users\ardo\Documents\personal\dev\notebook\test_count_1404\workspace --num_train_steps=100


In [22]:
!python C:\Users\ardo\Documents\personal\dev\notebook\test_count_1404\models\research\object_detection\model_main_tf2.py --model_dir= --pipeline_config_path=C:\Users\ardo\Documents\personal\dev\notebook\test_count_1404\workspace --num_train_steps=100

2021-04-15 14:13:59.347513: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library cudart64_101.dll
Traceback (most recent call last):
  File "C:\Users\ardo\Documents\personal\dev\notebook\test_count_1404\models\research\object_detection\model_main_tf2.py", line 32, in <module>
    from object_detection import model_lib_v2
  File "C:\Users\ardo\AppData\Roaming\Python\Python38\site-packages\object_detection\model_lib_v2.py", line 31, in <module>
    from object_detection import model_lib
  File "C:\Users\ardo\AppData\Roaming\Python\Python38\site-packages\object_detection\model_lib.py", line 34, in <module>
    from object_detection.builders import optimizer_builder
  File "C:\Users\ardo\AppData\Roaming\Python\Python38\site-packages\object_detection\builders\optimizer_builder.py", line 25, in <module>
    from official.modeling.optimization import ema_optimizer
ImportError: cannot import name 'ema_optimizer' from 'official.modeling.optimizatio